<a href="https://colab.research.google.com/github/smit-patel-m/SummerInternship/blob/main/PUF_Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, hamming_loss, f1_score
from sklearn.model_selection import train_test_split


In [10]:
import os
from google.colab import drive

if not os.path.exists("/content/db1"):
    drive.mount('/content/drive')
    !unzip "/content/drive/MyDrive/Summer Internship/ML Attack.zip" -d "/content/db1"
else:
    print("Folder '/content/db1' already exists. Skipping unzip.")


Folder '/content/db1' already exists. Skipping unzip.


# Load the Excel files into pandas DataFrames

In [11]:
# Load the Excel files into pandas DataFrames

response_delay_path = "/content/db1/ML Attack/Responce_Delay_2us.xlsx"
challenge_path = "/content/db1/ML Attack/Challenge.xlsx"
response_delay_sheet_names = pd.ExcelFile(response_delay_path).sheet_names
response_delay_df = pd.read_excel(response_delay_path, sheet_name=0,header=None)

challenge_df = pd.read_excel(challenge_path,header=None)


In [12]:
# Display the first few rows of each DataFrame to verify
print("Selected Sheet Name:", response_delay_sheet_names)

print("Response Delay DataFrame:")
display(response_delay_df.head())

print("\nChallenge DataFrame:")
display(challenge_df.head())

Selected Sheet Name: ['FD_MODE_1', 'VD_MODE_1', 'FD_MODE_2', 'VD_MODE_2', 'FD_MODE_3', 'VD_MODE_3', 'FD_MODE_4', 'VD_MODE_4', 'FD_MODE_5', 'VD_MODE_5', 'FD_MODE_6', 'VD_MODE_6', 'FD_MODE_7', 'VD_MODE_7', 'FD_MODE_8', 'VD_MODE_8']
Response Delay DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,1,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0,0,0,1,...,0,1,1,0,0,0,1,0,0,1
2,1,1,0,0,1,1,0,0,0,0,...,1,1,0,1,1,0,1,1,1,1
3,1,1,1,0,1,1,0,0,1,1,...,1,1,0,0,0,1,1,0,1,1
4,0,1,0,0,0,1,0,1,1,1,...,1,0,1,1,0,0,0,1,0,0



Challenge DataFrame:


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0,1,0,0,0,0,0,1,1,1,...,1,0,1,1,0,0,0,1,1,1
1,0,1,1,0,0,1,0,1,1,1,...,0,1,1,0,1,0,0,1,0,0
2,0,0,1,0,1,0,0,1,0,0,...,0,0,1,1,0,1,1,0,1,1
3,0,0,0,1,1,0,0,0,1,0,...,1,1,0,0,1,0,0,0,1,1
4,1,1,1,0,1,0,0,0,0,1,...,1,0,0,0,0,1,0,0,1,0


In [13]:
import os, re, joblib
import pandas as pd

def get_next_model_version(base_path="/content/drive/MyDrive/SummerIntern/Puf Models", prefix="puf_decision_tree"):
    os.makedirs(base_path, exist_ok=True)
    existing = [f for f in os.listdir(base_path) if f.startswith(prefix)]
    versions = [int(re.findall(r'V(\d+)', name)[0]) for name in existing if re.search(r'V\d+', name)]
    next_version = max(versions) + 1 if versions else 1
    model_folder = f"{prefix}_V{next_version}"
    return os.path.join(base_path, model_folder)

def save_dt_model(model, path, history=None):
    os.makedirs(path, exist_ok=True)

    model_path = os.path.join(path, "model.joblib")
    joblib.dump(model, model_path)
    print(f" Decision Tree model saved at: {model_path}")

    if history is not None:
        history_df = pd.DataFrame(history, columns=[
            "Max Depth", "Train Accuracy", "Val Accuracy",
            "Train Hamming Loss", "Val Hamming Loss", "Val Exact Match"
        ])
        excel_path = os.path.join(path, "training_history.xlsx")
        history_df.to_excel(excel_path, index=False)
        print(f" Training history saved at: {excel_path}")

def load_dt_model(path):
    model_path = os.path.join(path, "model.joblib")
    model = joblib.load(model_path)
    print(f" Decision Tree model loaded from: {model_path}")
    return model


In [14]:

X = challenge_df.values.astype(np.float32)
Y = response_delay_df.values.astype(np.float32)



In [15]:

X_temp, X_val, Y_temp, Y_val = train_test_split(X, Y, test_size=0.10, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X_temp, Y_temp, test_size=2/9, random_state=42)


In [16]:
from sklearn.metrics import accuracy_score

history = []
def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

for max_depth in range(1, 21):
    base_tree = DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=42)
    dt_model = MultiOutputClassifier(base_tree)
    dt_model.fit(X_train, Y_train)

    train_pred = dt_model.predict(X_train)
    val_pred = dt_model.predict(X_val)

    train_acc = accuracy_score(Y_train.flatten(), train_pred.flatten())
    val_acc = accuracy_score(Y_val.flatten(), val_pred.flatten())

    train_hamming = hamming_loss(Y_train, train_pred)
    val_hamming = hamming_loss(Y_val, val_pred)

    exact_val = exact_match_accuracy(Y_val, val_pred)

    print(f"\n[Depth = {max_depth}]")
    print(f" Train Accuracy (bitwise):      {train_acc:.4f}")
    print(f" Validation Accuracy (bitwise): {val_acc:.4f}")
    print(f" Train Hamming Loss:            {train_hamming:.4f}")
    print(f" Val Hamming Loss:              {val_hamming:.4f}")
    print(f" Exact Match (Val):             {exact_val:.4f}")

    history.append([
        max_depth,
        train_acc, val_acc,
        train_hamming, val_hamming,
        exact_val
    ])

    if train_acc == 1.0:
        print(f"\n✅ Achieved 100% training accuracy at depth {max_depth}!")
        break




[Depth = 1]
 Train Accuracy (bitwise):      0.5510
 Validation Accuracy (bitwise): 0.5536
 Train Hamming Loss:            0.4490
 Val Hamming Loss:              0.4464
 Exact Match (Val):             0.0000

[Depth = 2]
 Train Accuracy (bitwise):      0.5519
 Validation Accuracy (bitwise): 0.5437
 Train Hamming Loss:            0.4481
 Val Hamming Loss:              0.4563
 Exact Match (Val):             0.0000

[Depth = 3]
 Train Accuracy (bitwise):      0.5596
 Validation Accuracy (bitwise): 0.5379
 Train Hamming Loss:            0.4404
 Val Hamming Loss:              0.4621
 Exact Match (Val):             0.0000

[Depth = 4]
 Train Accuracy (bitwise):      0.5727
 Validation Accuracy (bitwise): 0.5296
 Train Hamming Loss:            0.4273
 Val Hamming Loss:              0.4704
 Exact Match (Val):             0.0000

[Depth = 5]
 Train Accuracy (bitwise):      0.5930
 Validation Accuracy (bitwise): 0.5191
 Train Hamming Loss:            0.4070
 Val Hamming Loss:              0.4809

In [17]:
# Save the model and training history
save_path = get_next_model_version(
    base_path="/content/drive/MyDrive/SummerIntern/Puf Models",
    prefix="puf_decision_tree"
)

save_dt_model(dt_model, save_path, history=history)


 Decision Tree model saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_decision_tree_V1/model.joblib
 Training history saved at: /content/drive/MyDrive/SummerIntern/Puf Models/puf_decision_tree_V1/training_history.xlsx


In [18]:
# Load saved model
load_path = "/content/drive/MyDrive/SummerIntern/Puf Models/puf_decision_tree_V1"
dt_model = load_dt_model(load_path)

# Evaluate
pred = dt_model.predict(X_test)

print("Test Accuracy (bitwise):", accuracy_score(Y_test.flatten(), pred.flatten()))
print("Test Hamming Loss:      ", hamming_loss(Y_test, pred))
print("Exact Match Accuracy:   ", exact_match_accuracy(Y_test, pred))


 Decision Tree model loaded from: /content/drive/MyDrive/SummerIntern/Puf Models/puf_decision_tree_V1/model.joblib
Test Accuracy (bitwise): 0.50065625
Test Hamming Loss:       0.49934375
Exact Match Accuracy:    0.0


In [20]:
dt_train_pred = dt_model.predict(X_train)
dt_test_pred = dt_model.predict(X_test)

print("\nDecision Tree Evaluation")
print(f" Train Bitwise Accuracy:  {accuracy_score(Y_train.flatten(), dt_train_pred.flatten()):.4f}")
print(f" Train F1 Score:          {f1_score(Y_train, dt_train_pred, average='macro'):.4f}")
print(f" Train Hamming Loss:      {hamming_loss(Y_train, dt_train_pred):.4f}")

print(f"\n Test Bitwise Accuracy:   {accuracy_score(Y_test.flatten(), dt_test_pred.flatten()):.4f}")
print(f" Test F1 Score:           {f1_score(Y_test, dt_test_pred, average='macro'):.4f}")
print(f" Test Hamming Loss:       {hamming_loss(Y_test, dt_test_pred):.4f}")
print(f" Exact Match (Test):      {exact_match_accuracy(Y_test, dt_test_pred):.4f}")


Decision Tree Evaluation
 Train Bitwise Accuracy:  1.0000
 Train F1 Score:          1.0000
 Train Hamming Loss:      0.0000

 Test Bitwise Accuracy:   0.5007
 Test F1 Score:           0.5406
 Test Hamming Loss:       0.4993
 Exact Match (Test):      0.0000


In [21]:
def exact_match_accuracy(y_true, y_pred):
    return np.mean(np.all(y_true == y_pred, axis=1))

dt_test_pred = dt_model.predict(X_test)
print("XGBoost Exact Match Test Accuracy:", exact_match_accuracy(Y_test, dt_test_pred))


XGBoost Exact Match Test Accuracy: 0.0


In [22]:
X_train_sample = X_train[:5]
Y_train_sample = Y_train[:5]
dt_pred_sample = dt_model.predict(X_train_sample)
bit_errors = np.sum(dt_pred_sample != Y_train_sample, axis=1)

print("\n Bit Error Count on First 5 Train Samples:")
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"  Input Challenge     : {np.round(X_train_sample[i], 2)}")
    print(f"  Target Response     : {Y_train_sample[i].astype(int)}")
    print(f"  Predicted Response  : {dt_pred_sample[i].astype(int)}")
    print(f"  Bit Errors          : {bit_errors[i]}")
    print("-" * 60)


 Bit Error Count on First 5 Train Samples:
Sample 1:
  Input Challenge     : [1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 0. 1.]
  Target Response     : [1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0]
  Predicted Response  : [1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0]
  Bit Errors          : 0
------------------------------------------------------------
Sample 2:
  Input Challenge     : [0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 0. 1.]
  Target Response     : [1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0]
  Predicted Response  : [1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 0]
  Bit Errors          : 0
------------------------------------------------------------
Sample 3:
  Input Challenge     : [1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0.]
  Target Response     :

In [23]:
X_test_sample = X_test[:5]
Y_test_sample = Y_test[:5]
dt_pred_sample = dt_model.predict(X_test_sample)
bit_errors = np.sum(dt_pred_sample != Y_test_sample, axis=1)

print("\n Bit Error Count on First 5 Test Samples:")
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"  Input Challenge     : {np.round(X_test_sample[i], 2)}")
    print(f"  Target Response     : {Y_test_sample[i].astype(int)}")
    print(f"  Predicted Response  : {dt_pred_sample[i].astype(int)}")
    print(f"  Bit Errors          : {bit_errors[i]}")
    print("-" * 60)


 Bit Error Count on First 5 Test Samples:
Sample 1:
  Input Challenge     : [1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 0. 1. 1. 1.]
  Target Response     : [1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 0]
  Predicted Response  : [1 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1]
  Bit Errors          : 11
------------------------------------------------------------
Sample 2:
  Input Challenge     : [1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 0.]
  Target Response     : [0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0]
  Predicted Response  : [0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1]
  Bit Errors          : 17
------------------------------------------------------------
Sample 3:
  Input Challenge     : [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 1.]
  Target Response     